In [19]:
from mpi4py import MPI
from petsc4py.PETSc import ScalarType
import numpy as np
import ufl
from dolfinx import fem, io, mesh, plot
from dolfinx.fem.petsc import LinearProblem
import dolfinx
import meshio
import gmsh
import time

In [20]:
start_time = time.time()

gmsh.initialize()
gmsh.clear()
gmsh.model.add("loaded_mesh")

gmsh.open("meshes/square/mesh.msh")

# gmsh.finalize()

Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Reading 'meshes/square/mesh.msh'...
Info    : 2017 nodes
Info    : 4032 elements
Info    : Done reading 'meshes/square/mesh.msh'


Explanation how to import mesh from GMsh:
https://jsdokken.com/FEniCS-workshop/src/external_mesh.html

In [21]:
# WARNING!!! It's important to specify that the mesh is 2D, otherwise it will perceive it as 3D
msh_data = dolfinx.io.gmsh.model_to_mesh(gmsh.model, MPI.COMM_SELF, 0, gdim=2)
msh = msh_data.mesh
cell_marker = msh_data.cell_tags
facet_marker = msh_data.facet_tags

# print(facet_marker.values)
# print(facet_marker.indices)
print(np.unique(facet_marker.values))

gmsh.finalize()

[1 2 3 4]


In [22]:
V = fem.functionspace(msh, ("Lagrange", 1))

In [23]:
# facets = mesh.locate_entities_boundary(
#     msh,
#     dim=(msh.topology.dim - 1),
#     marker=lambda x: np.isclose(x[0], 0.0) | np.isclose(x[0], 1.0),
# )

left_tag = 1
facets_left = facet_marker.find(left_tag)
right_tag = 2
facets_right = facet_marker.find(right_tag)


dofs_left = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets_left)
dofs_right = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets_right)

bc_left = fem.dirichletbc(value=ScalarType(0), dofs=dofs_left, V=V)
bc_right = fem.dirichletbc(value=ScalarType(0), dofs=dofs_right, V=V)

In [24]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
x = ufl.SpatialCoordinate(msh)
# f = 10 * ufl.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / 0.02)
g = 10 * ufl.sin(6 * x[0])
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
# L = ufl.inner(f, v) * ufl.dx + ufl.inner(g, v) * ufl.ds

L = ufl.inner(g, v) * ufl.ds

In [25]:
# There is a more manual method to solve this, similar to NGSolve, Heat_Equation_fenics.ipynb.

problem = LinearProblem(
    a,
    L,
    bcs=[bc_left, bc_right],
    petsc_options_prefix="demo_poisson_",
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "ksp_error_if_not_converged": True},
)
uh = problem.solve()
assert isinstance(uh, fem.Function)

end_time = time.time()

print(f"Runtime: {np.round(end_time - start_time, 3)} s")

Runtime: 0.047 s


In [26]:
# with io.XDMFFile(msh.comm, "output/square/poisson_sqaure_fenics.xdmf", "w") as file:
#     file.write_mesh(msh)
#     file.write_function(uh)

# with io.VTKFile(msh.comm, "output/square/poisson_sqaure_fenics.pvd", "w") as file:
#     file.write_mesh(msh)
#     file.write_function(uh, t=0.0)

Evaluating function at points: https://jsdokken.com/FEniCS-workshop/src/deep_dive/expressions.html#interpolation-on-a-subset-of-cells

In [27]:
import pyvista as pv

res = pv.read("meshes/square/mesh.msh")
points = res.points

bb_tree = dolfinx.geometry.bb_tree(msh, msh.topology.dim, padding=1e-10)
potential_colliding_cells = dolfinx.geometry.compute_collisions_points(bb_tree, points)

colliding_cells = dolfinx.geometry.compute_colliding_cells(msh, potential_colliding_cells, points)

points_on_proc = []
cells = []
for i, point in enumerate(points):
    if len(colliding_cells.links(i)) > 0:
        points_on_proc.append(point)
        cells.append(colliding_cells.links(i)[0])
points_on_proc = np.array(points_on_proc, dtype=np.float64).reshape(-1, 3)
cells = np.array(cells, dtype=np.int32)
sol = uh.eval(points_on_proc, cells)

# print(np.size(sol.flatten()))

res["sol"] = sol

res.save("output/square/poisson_sqaure_fenics.vtu")